In [ ]:
import pandas as pd
import numpy as np
import random
import time
from tqdm import tqdm
from google.cloud import bigquery

# Set options to display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

### Dataset Creation
* For each client that has gone through the workbook, we have already gotten their plans and then matched those plans with the plans from the Ideon dataset. This has been done manually. Creating the dataset will be one of the more difficult tasks.
* For each client, we want to go get all the available plans, create a dataframe, match only the matched plans in a dataset, and then keep doing that for all clients.
* After (or maybe during) we then need to create the false match case, where we still need to store all of the plans for a given client and then create a false match in the dataset.
* For each plan should we have every plan being a false match? How should we create the dataset?

In [ ]:
# Reading in small group plans actuals to begin to build dataset
small_group_actuals_file_path = '~/match-plans/data/small_group_plans_actuals.csv'
small_group_actuals_df = pd.read_csv(small_group_actuals_file_path)
small_group_actuals_df['plan_effective_start_date'] = pd.to_datetime(small_group_actuals_df['plan_effective_start_date'])
small_group_actuals_df.rename(columns={'plan_id':'id','carrier_name':'census_carrier_name'},inplace=True)


# prep the data to be able to query and iterate through
grouped = small_group_actuals_df.groupby(['client_name', 'zip_code', 'plan_effective_start_date'])

# Initialize list to store client objects
clients = []

# Loop through each group
for (client_name, zip_code, start_date), group in grouped:
    client = {}
    client['client_name'] = client_name
    client['zip_code'] = zip_code
    client['plan_effective_start_date'] = start_date

    # Extract year and quarter
    year = start_date.year
    quarter = (start_date.month - 1) // 3 + 1
    
    client['year'] = year
    client['quarter'] = f'Q{quarter}'
    client['plans'] = group.drop(columns=['client_name', 'zip_code', 'plan_effective_start_date']).to_dict(orient='records')
    clients.append(client)

print(clients[0])

In [ ]:
# BigQuery Client to Query Dataset
GOOGLE_APPLICATION_CREDENTIALS = '/Users/kieranshaw/.config/gcloud/application_default_credentials.json'

# initialize the client and saved dataframe
bq_client = bigquery.Client()
df = pd.DataFrame()
start_time = time.time()

for client in tqdm(clients, desc='Processing clients'):
    query_start_time = time.time()

    query = f"""
        WITH SelectedZip AS (
            SELECT * 
            FROM airbyte_ideon.zip_counties
            WHERE zip_code_id = '{client['zip_code']}'
            AND _ab_source_file_url LIKE '%CA/{client['year']}/{client['quarter']}%'
        )
        SELECT
            p.*
        FROM 
            SelectedZip zc
        JOIN 
            airbyte_ideon.plan_counties pc 
            ON zc.county_id = pc.county_id
            AND pc._ab_source_file_url LIKE '%CA/{client['year']}/{client['quarter']}%'
        JOIN 
            airbyte_ideon.plans p 
            ON p.id = pc.plan_id
            AND p._ab_source_file_url LIKE '%CA/{client['year']}/{client['quarter']}%'
    """
    
    # Run the query
    query_job = bq_client.query(query)

    # Fetch the results
    results = query_job.result()

    # Convert the results to a pandas DataFrame
    query_df = results.to_dataframe()

    # Create a DataFrame from the 'plans' key in the client object
    client_plans_df = pd.DataFrame(client['plans'])

    # Add columns for the client info
    client_plans_df['client_name'] = client['client_name']
    client_plans_df['zip_code'] = client['zip_code']
    client_plans_df['plan_effective_start_date'] = client['plan_effective_start_date']

    # Merge the results with the client info
    match_df = pd.merge(client_plans_df, query_df, on='id')
    match_df['is_match'] = 1

    # NON MATCHING PLANS FAKE DATA
    non_matching_plans = query_df[~query_df['id'].isin(client_plans_df['id'])]

    # Create a DataFrame for non-matching plans with client info
    non_matching_df = non_matching_plans.copy()
    non_matching_df['client_name'] = client['client_name']
    non_matching_df['zip_code'] = client['zip_code']
    non_matching_df['plan_effective_start_date'] = client['plan_effective_start_date']
    non_matching_df['is_match'] = 0  # Label as non-match
    non_matching_df['plan_admin_name'] = np.NaN
    non_matching_df['census_carrier_name'] = np.NaN

    # Add random plan_admin_name and carrier_name from the client's set
    client_plan_admin_names = set(client_plans_df['plan_admin_name'].dropna())
    client_carrier_names = set(client_plans_df['census_carrier_name'].dropna())
    non_matching_df['plan_admin_name'] = non_matching_df.apply(lambda row: random.choice(list(client_plan_admin_names)) if pd.isnull(row['plan_admin_name']) else row['plan_admin_name'], axis=1)
    non_matching_df['census_carrier_name'] = non_matching_df.apply(lambda row: random.choice(list(client_carrier_names)) if pd.isnull(row['census_carrier_name']) else row['census_carrier_name'], axis=1)

    final_df = pd.concat([match_df, non_matching_df], ignore_index=True)
    df = pd.concat([df, final_df], ignore_index=True)
    query_end_time = time.time()

end_time = time.time()
print(f"Processed {len(clients)} clients in {end_time - start_time:.2f} seconds")


In [ ]:
df.to_csv('~/match-plans/data/small_group_dataset.csv',index=False)